In [2]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

In [3]:
BASE_PATH = '../almrrc2021/almrrc2021-data-training/model_build_inputs/'

In [4]:
# Loading all the package data
package_data = json.load(open(BASE_PATH + '/package_data.json'))

In [5]:
# Flattens the json object, essentially every package gets a row (unless the package had multiple delivery attempts)
# If you don't want to load it all, you can use the RouteID as a key on the package_data dictionary
flattened_package_data = []
for route_id, stops in package_data.items():
    for stop_id, packages in stops.items():
        for package_id, details in packages.items():
            record = {
                "route_id": route_id,
                "stop_id": stop_id,
                "package_id": package_id,
                "scan_status": details["scan_status"] if "scan_status" in details else None, # note model_apply_inputs does not have scan_status, model_build_inputs does
                "start_time_utc": details["time_window"]["start_time_utc"],
                "end_time_utc": details["time_window"]["end_time_utc"],
                "planned_service_time_seconds": details["planned_service_time_seconds"],
                "depth_cm": details["dimensions"]["depth_cm"],
                "height_cm": details["dimensions"]["height_cm"],
                "width_cm": details["dimensions"]["width_cm"],
            }
            flattened_package_data.append(record)

package_data_df = pd.DataFrame(flattened_package_data)

In [6]:
route_data = json.load(open(BASE_PATH + '/route_data.json'))

In [7]:

flattened_route_data = []

for route_id, info in route_data.items():
    for stop_id, stop_details in info['stops'].items():
        flattened_route_data.append({
            "route_id": route_id,
            "station_code": info['station_code'],
            "date": info['date_YYYY_MM_DD'],
            "departure_time_utc": info['departure_time_utc'],
            "executor_capacity_cm3": info['executor_capacity_cm3'],
            "route_score": info['route_score'],
            "stop_id": stop_id,
            "lat": stop_details['lat'],
            "lng": stop_details['lng'],
            "type": stop_details['type'],
            "zone_id": stop_details['zone_id']
        })

route_data_df = pd.DataFrame(flattened_route_data)

In [8]:
# Loading all the travel times (took around 70 seconds for me)
vehicle_travel_times = json.load(open(BASE_PATH + '/travel_times.json'))

In [9]:
actual_sequences = json.load(open(BASE_PATH + '/actual_sequences.json'))

proposed_sequences = json.load(open('proposed_sequences.json'))


In [10]:
def sequence_to_stops_for_route(route_id, actual):
    if actual:
        route_sequence = actual_sequences[route_id]['actual']
    else:
        route_sequence = proposed_sequences[route_id]['proposed']
    
    # Sorting them by their order
    route_sequence_sorted = dict(sorted(route_sequence.items(), key=lambda item: item[1]))

    # Converting keys to an array
    route_sequence_sorted = list(route_sequence_sorted.keys())

    # The last stop to the station (first stop) is not included in the sequence so we will manually add it
    route_sequence_sorted.append(route_sequence_sorted[0])

    return route_sequence_sorted

In [11]:
# VRP Event Types

# Both
ARRIVE = 'ARRIVE'
DEPART = 'DEPART'

# Vehicle
SERVICE_START = 'SERVICE_START'
SERVICE_END = 'SERVICE_END'

In [12]:
def get_data_for_route(route_id):
    package_data = package_data_df[package_data_df['route_id'] == route_id]
    route_data = route_data_df[route_data_df['route_id'] == route_id]
    travel_times_route = vehicle_travel_times[route_id]

    package_info_agg = package_data.groupby('stop_id').apply(
    lambda x: pd.Series({
        'packages': x[['package_id', 'depth_cm', 'height_cm', 'width_cm']].to_dict('records'),
        'num_packages': len(x),
        'service_time_seconds': x['planned_service_time_seconds'].mean() # Not sure if this should be sum or average. https://github.com/MIT-CAVE/rc-cli/blob/main/templates/data_structures.md
    })
    ).reset_index()

    route_data_route_df = pd.merge(route_data, package_info_agg, how='left', on='stop_id')

    return package_data.copy(), route_data_route_df.copy(), travel_times_route.copy()

In [13]:
class Event:
  def __init__(self, event_type, stop_id, time, data): # data holds extra information, like coordinates
    self.event_type = event_type
    self.stop_id = stop_id
    self.time = time
    self.data = data

  def __str__(self):
    return f"{self.event_type} {self.stop_id} {round(self.time, 1)} {self.data}"

  def to_dict(self):
    # Convert the event to a dictionary
    return {
        'event_type': self.event_type,
        'stop_id': self.stop_id,
        'time': self.time,
        'coordinates': self.data,
    }

In [14]:
class VRPHelper():
    def __init__(self, route_id, sequence, package_data, route_data, vehicle_travel_times):
        self.route_id = route_id
        self.sequence = sequence
        self.package_data = package_data
        self.route_data = route_data
        self.vehicle_travel_times = vehicle_travel_times

        self.vehicle_events = []
        self.vehicle_path = []
        self.final_time = 0
    
    def generate_events_path_time(self):
        cur_coords = None
        cur_stop = None
        cur_time = 0

        # First stop is the station
        cur_stop = self.sequence[0]
        stop = self.route_data[self.route_data['stop_id'] == cur_stop].iloc[0]
        cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}

        first_event = Event(DEPART, cur_stop, cur_time, cur_coords)
        self.vehicle_events.append(first_event)
        self.vehicle_path.append(cur_stop)

        for i in range(len(self.sequence)):
            # Travel there
            cur_time += self.vehicle_travel_times[self.sequence[i]][self.sequence[i + 1]]
            # Arrive
            cur_stop = self.sequence[i + 1]
            stop = self.route_data[self.route_data['stop_id'] == cur_stop].iloc[0]
            cur_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
            arrive_event = Event(ARRIVE, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(arrive_event)
            self.vehicle_path.append(cur_stop)

            # We have finished delivering and arrived at the station
            if cur_stop == self.sequence[-1]:
                break

            # Driver starts service by delivering package
            service_start_event = Event(SERVICE_START, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(service_start_event)
            cur_time += stop['service_time_seconds']

            # Driver finishes service of delivering package
            service_end_event = Event(SERVICE_END, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(service_end_event)

            # Driver departs
            depart_event = Event(DEPART, cur_stop, cur_time, cur_coords)
            self.vehicle_events.append(depart_event)
        
        # Route is now finished
        self.final_time = cur_time
        
        return self.vehicle_events, self.vehicle_path, self.final_time
    def print_vehicle_path(self):
          print(self.vehicle_path)  # Ensure vehicle_path is correctly converted to string

    def print_vehicle_events(self):
          for event in self.vehicle_events:
              print(event)  # Convert Event object to string


    def save_vehicle_path(self):
        path_json = []
        for i, stop_id in enumerate(self.vehicle_path):
            stop = self.route_data[self.route_data['stop_id'] == stop_id].iloc[0]
            vehicle_coords = {'lat' : stop['lat'], 'lng' : stop['lng']}
            path_json.append({"id": i+1, "stop": stop_id, "coordinates": vehicle_coords})

        with open(f"{self.route_id}_vehicle_path.json", 'w') as file:
            json.dump(path_json, file, indent=4)

    def save_vehicle_events(self):
        # Convert the list of events to a list of dictionaries and assign an ID to each
        events_with_ids = [{'id': i+1, **event.to_dict()} for i, event in enumerate(self.vehicle_events)]
        # Save the list of events as a JSON file
        with open(f"{self.route_id}_vehicle_events.json", 'w') as file:
            json.dump(events_with_ids, file, indent=4)

In [15]:
proposed_route_ids = list(proposed_sequences.keys())

route_id = proposed_route_ids[0]
package_data_route_df, route_data_route_df, vehicle_travel_times_route = get_data_for_route(route_id)

# Toggle actual if you want the route to be chosen from the actual sequences or the generated proposed sequences
sequence = sequence_to_stops_for_route(route_id, actual=False)

In [16]:
driver = VRPHelper(route_id, sequence, package_data_route_df, route_data_route_df, vehicle_travel_times_route)
vehicle_events, vehicle_path, final_time = driver.generate_events_path_time()

In [17]:
driver.final_time

27402.09999999999